<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In terminal:  
\\$ source activate tensorflow_p36  
\\$ pip install --update scikit-image  
Restart kernel for notebook  

In [38]:
from skimage import io, transform
from skimage.transform import resize
import numpy as np
import os

In [39]:
!pwd
forest_path = './data/forest/'
mountain_path = './data/mountain/'

/home/ec2-user/SageMaker/DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks


In [40]:
forest_list = os.listdir(forest_path)
mountain_list = os.listdir(mountain_path)

In [41]:
len(mountain_list), len(forest_list)

(375, 329)

In [42]:
def create_arrays(path, images_list, reduce_size=4):
    x = []
    for i in images_list:
        if i[-3:] == 'jpg':
            image = io.imread(path + i)
            resized = resize(image, (image.shape[0] // reduce_size, image.shape[1] // reduce_size),
                       anti_aliasing=True)
            x.append(resized)
    out = np.array(x)
    return out

In [43]:
forests= create_arrays(forest_path, forest_list)
mountains = create_arrays(mountain_path, mountain_list)
forests.shape, mountains.shape

((328, 64, 64, 3), (374, 64, 64, 3))

In [46]:
forest_labels = np.zeros(328, dtype=int)
mountain_labels = np.ones(374, dtype=int)
forest_labels.shape, mountain_labels.shape

((328,), (374,))

In [47]:
full_images = np.vstack((forests, mountains))
full_labels = np.hstack((forest_labels, mountain_labels))
full_images.shape, full_labels.shape

((702, 64, 64, 3), (702,))

In [48]:
full_images[0].shape

(64, 64, 3)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(full_images, full_labels, test_size=.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Instatiate Model

In [49]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [50]:
for layer in resnet.layers:
    layer.trainable = False

In [51]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [14]:
# DON'T RUN THIS CELL IF YOU DON'T WANT HUGE OUTPUT
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
______________________________________________________________________________________________

In [52]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Fit Model

In [53]:
model.fit(full_images, full_labels, epochs=5, validation_split=.25)

Train on 526 samples, validate on 176 samples
Epoch 1/5
526/526 [==============================] - 16s 31ms/sample - loss: 0.4212 - acc: 0.8346 - val_loss: 0.2925 - val_acc: 1.0000
Epoch 2/5
526/526 [==============================] - 10s 18ms/sample - loss: 0.1305 - acc: 0.9563 - val_loss: 0.1012 - val_acc: 1.0000
Epoch 3/5
526/526 [==============================] - 9s 17ms/sample - loss: 0.0741 - acc: 0.9810 - val_loss: 0.0924 - val_acc: 1.0000
Epoch 4/5
526/526 [==============================] - 9s 17ms/sample - loss: 0.0343 - acc: 0.9886 - val_loss: 0.1218 - val_acc: 1.0000
Epoch 5/5
526/526 [==============================] - 9s 17ms/sample - loss: 0.0325 - acc: 0.9924 - val_loss: 0.1852 - val_acc: 1.0000


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [55]:
full_images.shape, full_labels.shape

((702, 64, 64, 3), (702,))

In [59]:
model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(full_images[0].shape)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 12, 64)        36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 64)               

In [60]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [61]:
model.fit(full_images, full_labels, epochs=3, validation_split=.25)

Train on 526 samples, validate on 176 samples
Epoch 1/3
526/526 [==============================] - 3s 6ms/sample - loss: 9.5614 - acc: 0.3764 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/3
526/526 [==============================] - 1s 3ms/sample - loss: 9.5614 - acc: 0.3764 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/3
526/526 [==============================] - 1s 3ms/sample - loss: 9.5614 - acc: 0.3764 - val_loss: 0.0000e+00 - val_acc: 1.0000


# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [ ]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language